In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

<div class="alert alert-block alert-info">
<b>To do</b>:

 - Mapping
 - Encoding
 - Algo
    
Subsidiary questions: Product Life cycel status (RaMPDOWN)
</div>

In [2]:
X = pd.read_csv('datasets_hi4/train-data.csv', sep=';')
X.drop_duplicates(inplace=True, ignore_index=True)

In [3]:
X['Reference proxy'] = X['Reference proxy'].apply(lambda x: x.replace('reference-', ''))
X['Product  Line proxy'] = X['Product  Line proxy'].apply(lambda x: x.replace('Product Line-', ''))
X['Division proxy'] = X['Division proxy'].apply(lambda x: x.replace('Division-', ''))
X['Customer Persona proxy'] = X['Customer Persona proxy'].apply(lambda x: x.replace('Customer Segmentation-', ''))
X['Strategic Product Family proxy'] = X['Strategic Product Family proxy'].apply(lambda x: x.replace('Strategic Product Family-', ''))

In [4]:
X[['Quarter', 'Year']] = X.Date.str.split(expand=True)
X['Quarter'] = X['Quarter'].map({'jan-apr': 1, 'may-jul': 2, 'may-aug': 2, 'sep-dec': 3})
X['Year'] = X['Year'].astype(int)
X['Quarter_cumuled'] = (X['Quarter']-2) + (X['Year']-2020)*3
X.drop('Date', axis=1, inplace=True)

In [5]:
X['Site'] = X['Site'].map({'OOS_DRP': 'Unknown_Unknown_OOS_DRP'}).fillna(X['Site'])
site_det = X['Site'].str.split('_', expand=True)
site_det[2] = pd.Series(np.where(site_det[3].notna(), site_det[2]+' '+ site_det[3], site_det[2]))
X[['Site_country', 'Site_type', 'Site_complementary']] = site_det[[0, 1, 2]]
X.drop('Site', axis=1, inplace=True)

In [6]:
X['Month 1'].fillna(X['Month 2'], inplace=True)
X['Product Life cycel status'] = X['Product Life cycel status'].map({'NPI': 3, 'EOL': 0, 'RaMPDOWN': 1}).fillna(2)

In [7]:
X.columns

Index(['index', 'id_product', 'Region', 'Country', 'Operations', 'Zone',
       'Cluster', 'Reference proxy', 'Product  Line proxy', 'Division proxy',
       'Customer Persona proxy', 'Strategic Product Family proxy',
       'Product Life cycel status', 'Month 1', 'Month 2', 'Month 3', 'Month 4',
       'Quarter', 'Year', 'Quarter_cumuled', 'Site_country', 'Site_type',
       'Site_complementary'],
      dtype='object')

In [8]:
X[X.id_product==149445]

,index,id_product,Region,Country,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Month 1,Month 2,Month 3,Month 4,Quarter,Year,Quarter_cumuled,Site_country,Site_type,Site_complementary
5,149445,149445,EUROPE,ES,Europe Operations,Iberia,Spain,7529,4,3,3,9,3.0,0,0,0,0,3,2020,1,ES,DC,IberiaSantBoi
229671,2108301,149445,EUROPE,ES,Europe Operations,Iberia,Spain,7529,4,3,3,9,3.0,0,0,0,3,2,2023,9,ES,DC,IberiaSantBoi
381702,1863444,149445,EUROPE,ES,Europe Operations,Iberia,Spain,7529,4,3,3,9,3.0,0,0,0,0,1,2023,8,ES,DC,IberiaSantBoi
706951,639159,149445,EUROPE,ES,Europe Operations,Iberia,Spain,7529,4,3,3,9,3.0,0,0,0,0,2,2021,3,ES,DC,IberiaSantBoi
831837,1128873,149445,EUROPE,ES,Europe Operations,Iberia,Spain,7529,4,3,3,9,3.0,0,0,0,0,1,2022,5,ES,DC,IberiaSantBoi
1043996,1618587,149445,EUROPE,ES,Europe Operations,Iberia,Spain,7529,4,3,3,9,3.0,0,0,0,4,3,2022,7,ES,DC,IberiaSantBoi
1638244,884016,149445,EUROPE,ES,Europe Operations,Iberia,Spain,7529,4,3,3,9,3.0,0,0,0,0,3,2021,4,ES,DC,IberiaSantBoi


In [9]:
X.head()

,index,id_product,Region,Country,Operations,Zone,Cluster,Reference proxy,Product Line proxy,Division proxy,Customer Persona proxy,Strategic Product Family proxy,Product Life cycel status,Month 1,Month 2,Month 3,Month 4,Quarter,Year,Quarter_cumuled,Site_country,Site_type,Site_complementary
0,645874,156160,EUROPE,DE,Europe Operations,DACH,Germany,13523,4,3,3,12,2.0,0,0,0,0,2,2021,3,NL,DC,Venray
1,469488,224631,CHINA,CN,China Operations,China & HK,China,12778,4,3,14,9,2.0,0,1,0,0,1,2021,2,CN,DC,Shanghai
2,348904,104047,EUROPE,GB,Europe Operations,UK and Ireland,United Kingdom,3513,3,2,23,7,2.0,0,2,0,0,1,2021,2,NL,DC,HLD
3,1725822,11823,EAJP,AU,International Operations,Pacific,Australia,672,1,1,11,3,2.0,0,0,0,0,1,2023,8,AU,DC,Perth
4,404781,159924,EUROPE,ES,Europe Operations,Iberia,Spain,3496,3,2,14,7,2.0,0,0,0,0,1,2021,2,NL,DC,HLD


In [10]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1762970 entries, 0 to 1762969
Data columns (total 23 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   index                           int64  
 1   id_product                      int64  
 2   Region                          object 
 3   Country                         object 
 4   Operations                      object 
 5   Zone                            object 
 6   Cluster                         object 
 7   Reference proxy                 object 
 8   Product  Line proxy             object 
 9   Division proxy                  object 
 10  Customer Persona proxy          object 
 11  Strategic Product Family proxy  object 
 12  Product Life cycel status       float64
 13  Month 1                         object 
 14  Month 2                         object 
 15  Month 3                         object 
 16  Month 4                         object 
 17  Quarter                    

In [11]:
#string_columns = X.select_dtypes(include=['object']).columns

#for col in string_columns:
#    le = LabelEncoder()
#    X[col] = le.fit_transform(X[col])

In [12]:
#matrice_corr = np.abs(X.corr())
#masque_triangulaire = np.tril(np.ones(matrice_corr.shape), k=-1).astype(bool)
#matrice_corr_triangulaire = matrice_corr.where(masque_triangulaire)

In [13]:
#plt.figure(figsize=(8, 6))
#sns.heatmap(matrice_corr_triangulaire, cmap='coolwarm', mask=~masque_triangulaire)
#plt.title('Matrice de corrélation en valeurs absolues (sous forme triangulaire)')
#plt.show()